In [ ]:
import pandas as pd
import numpy as np
import networkx as nx

import sys
sys.path.insert(1, '../rtsvg')

from rtsvg import *

import time
import random

from math import log

from IPython.display import Javascript, HTML, display
%matplotlib inline

In [ ]:
rt = RACETrack()

In [ ]:
tups = [ ('a', 'b',   2),
         ('b', 'c',   2),
         ('c', 'd',   2),
         ('d', 'e',   2),
         ('a', 'a0', 10),
         ('a', 'a1', 11),
         ('a', 'a2', 12),
         ('c', 'c0',  8),
         ('c', 'c1',  9),
         ('c1','a1', 50)
       ]
fms,tos,weights = [],[],[]
for tup in tups:
    fms.    append(tup[0])
    tos.    append(tup[1])
    weights.append(tup[2])
for i in range(0,30):
    fms.    append('d')
    tos.    append('d'+str(i))
    weights.append(0.1)
for i in range(0,15):
    fms.    append('e')
    tos.    append('e'+str(i))
    weights.append(0.3)
for i in range(0,10):
    fms.    append('b')
    tos.    append('b'+str(i))
    weights.append(0.5)

df = pd.DataFrame({'fm':fms,'to':tos,'weight':weights})

In [ ]:
relates = [('fm','to')]
pos1 = rt.springLayout(rt.createNetworkXGraph(df, relates),spring_exp=.1)
pos2 = rt.springLayout(rt.createNetworkXGraph(df, relates),spring_exp=.2)
pos3 = rt.springLayout(rt.createNetworkXGraph(df, relates),spring_exp=.3)

pos4 = rt.springLayout(rt.createNetworkXGraph(df, relates),spring_exp=.5)
pos5 = rt.springLayout(rt.createNetworkXGraph(df, relates),spring_exp=.6)
pos6 = rt.springLayout(rt.createNetworkXGraph(df, relates),spring_exp=.7)

pos7 = rt.springLayout(rt.createNetworkXGraph(df, relates),spring_exp=1)
pos8 = rt.springLayout(rt.createNetworkXGraph(df, relates),spring_exp=2)
pos9 = rt.springLayout(rt.createNetworkXGraph(df, relates),spring_exp=3)

spec = {    '1,1':  ('linkNode',{'pos':pos1, 'relationships':relates, 'draw_labels':False}),
            '2,1':  ('linkNode',{'pos':pos2, 'relationships':relates, 'draw_labels':False}),
            '3,1':  ('linkNode',{'pos':pos3, 'relationships':relates, 'draw_labels':False}),
            '1,2':  ('linkNode',{'pos':pos4, 'relationships':relates, 'draw_labels':False}),
            '2,2':  ('linkNode',{'pos':pos5, 'relationships':relates, 'draw_labels':False}),
            '3,2':  ('linkNode',{'pos':pos6, 'relationships':relates, 'draw_labels':False}),
            '1,3':  ('linkNode',{'pos':pos7, 'relationships':relates, 'draw_labels':False}),
            '2,3':  ('linkNode',{'pos':pos8, 'relationships':relates, 'draw_labels':False}),
            '3,3':  ('linkNode',{'pos':pos9, 'relationships':relates, 'draw_labels':False}) }

# svg = rt.linkNode(df, relates, pos, draw_labels=False)
svg = rt.multiWidgetPanel(df, spec, w=768, h=512)

display(HTML(svg))

In [ ]:
relates = [('fm','to')]
df['inv_log_weight'] = df['weight'].apply(lambda x: 1.0 / (log(x+1)+1))
pos = rt.springLayout(rt.createNetworkXGraph(df, relates, count_by='inv_log_weight'), use_weights=True, spring_exp=0.1)
svg = rt.linkNode(df, relates, pos, w=512,h=512)
display(HTML(svg))

In [ ]:
#
# Create a synthetic graph
# ... will be "num_nodes" + 1 nodes...
# ... num_edges is on top of all the edges to connect
#     the nodes in a single path...
#
def createSyntheticGraph(num_nodes, num_edges):
    fms = []
    tos = []
    for i in range(0,num_nodes):
        fms.append(str(i))
        tos.append(str(i+1))
    for i in range(0,num_edges):
        fms.append(str(random.randint(0,num_nodes)))
        tos.append(str(random.randint(0,num_nodes)))
    return pd.DataFrame({'fm':fms,'to':tos})

relates = [('fm','to')]
df = createSyntheticGraph(50,100)
pos = rt.springLayout(rt.createNetworkXGraph(df, relates), spring_exp=0.5)
svg = rt.linkNode(df, relates, pos, w=128, h=128, draw_labels=False)
display(HTML(svg))

In [ ]:
num_nodes_ls  = []
time_taken_ls = []
method_ls     = []
for num_nodes in range(10,160,10):
    print(num_nodes)
    df = createSyntheticGraph(num_nodes, int(num_nodes*1.2))
    relates = [('fm','to')]
    ts0 = time.time()
    pos = rt.springLayout(rt.createNetworkXGraph(df, relates), spring_exp=0.5)
    ts1 = time.time()
    num_nodes_ls.  append(num_nodes)
    time_taken_ls. append(ts1-ts0)
    method_ls.     append('reference')

    ts2 = time.time()
    nx.all_pairs_shortest_path_length(rt.createNetworkXGraph(df, relates))
    ts3 = time.time()
    num_nodes_ls.  append(num_nodes)
    time_taken_ls. append(ts3-ts2)
    method_ls.     append('all_pairs')

    ts4 = time.time()
    nx.spring_layout(rt.createNetworkXGraph(df, relates))
    ts5 = time.time()
    num_nodes_ls.  append(num_nodes)
    time_taken_ls. append(ts5-ts4)
    method_ls.     append('nx_springs')

    # too horrible to include
    #ts6 = time.time()
    #pos = rt.springLayoutThreaded(rt.createNetworkXGraph(df, relates), spring_exp=0.5)
    #ts7 = time.time()
    #num_nodes_ls.  append(num_nodes)
    #time_taken_ls. append(ts7-ts6)
    #method_ls.     append('threaded')


In [ ]:
df = pd.DataFrame({'num_nodes':num_nodes_ls, 'time':time_taken_ls, 'method':method_ls})
display(HTML(rt.xy(df, 'num_nodes', 'time', color_by='method', dot_size='large', w=256, h=256)))

In [ ]:
display(HTML(rt.histogram(df,bin_by='method',color_by='method',count_by='time',w=512,h=160)))